#### taxi_id_number 生成器
生成 num_taxis 个唯一的出租车 ID，且每个 ID 是随机的

In [1]:
import numpy as np

def taxi_id_number(num_taxis):
    arr = np.arange(num_taxis)
    np.random.shuffle(arr)
    for i in range(num_taxis):
        yield arr[i]    # 每次调用 next() 时，生成器从上次暂停的位置继续执行，返回当前的出租车 ID，直到没有更多的 ID 为止


#### shift_info 生成器
生成每个出租车的班次信息，包括班次的开始时间、结束时间和预计的平均旅行次数

In [2]:
def shift_info():           # 分时间0, 8, 16; 并且不同时间段内，平均接客次数也是不同的
    start_times_and_freqs = [(0, 8), (8, 30), (16, 15)]
    indices = np.arange(len(start_times_and_freqs))
    while True:                         # 在白天安排了较多的出租车，而为傍晚或整宿的班次安排了较少的出租车
        idx = np.random.choice(indices, p=[0.25, 0.5, 0.25])
        start = start_times_and_freqs[idx]
        yield (start[0], start[0] + 7.5, start[1])  # 返回开始, 结束时间 以及 接客数


#### taxi_process 生成器
模拟出租车的工作流程，包括接乘客、送乘客等事件，直到班次结束

使用**泊松分布**来模拟事件之间的时间间隔（`np.random.poisson()`）


In [4]:
from dataclasses import dataclass

@dataclass
class TimePoint:
    taxi_id: int
    name: str
    time: float

    def __lt__(self, other):
        return self.time < other.time

In [5]:
def taxi_process(taxi_id_generator, shift_info_generator):
    taxi_id = next(taxi_id_generator)
    shift_start, shift_end, shift_mean_trips = next(shift_info_generator)
    # 正态分布模拟接客数
    actual_trips = round(np.random.normal(loc=shift_mean_trips, scale=2))
    average_trip_time = 6.5 / shift_mean_trips * 60  # 将平均行程时长的单位转换为分钟
    between_events_time = 1.0 / (shift_mean_trips - 1) * 60  # 出租车的利用率较高
    
    time = shift_start
    yield TimePoint(taxi_id, 'start shift', time)
    
    deltaT = np.random.poisson(between_events_time) / 60
    time += deltaT
    
    for i in range(actual_trips):
        yield TimePoint(taxi_id, 'pick up    ', time)
        deltaT = np.random.poisson(average_trip_time) / 60
        time += deltaT
        yield TimePoint(taxi_id, 'drop off   ', time)
        deltaT = np.random.poisson(between_events_time) / 60
        time += deltaT
    
    deltaT = np.random.poisson(between_events_time) / 60
    time += deltaT
    yield TimePoint(taxi_id, 'end shift  ', time)


#### Simulator 类
控制整个模拟流程，生成并排序所有出租车的事件，然后逐步输出事件

In [6]:
import queue

class Simulator:
    def __init__(self, num_taxis):
        self._time_points = queue.PriorityQueue()
        taxi_id_generator = taxi_id_number(num_taxis)
        shift_info_generator = shift_info()
        self._taxis = [
            taxi_process(taxi_id_generator, shift_info_generator) for i in range(num_taxis)
        ]
        self._prepare_run()

    def _prepare_run(self):
        for t in self._taxis:
            while True:
                try:
                    e = next(t)
                    self._time_points.put(e)
                except:
                    break

    def run(self):
        sim_time = 0
        while sim_time < 24:
            if self._time_points.empty():
                break
            p = self._time_points.get()
            sim_time = p.time
            print(p)



In [7]:
sim = Simulator(1000)
sim.run()

TimePoint(taxi_id=np.int64(961), name='start shift', time=0)
TimePoint(taxi_id=np.int64(464), name='start shift', time=0)
TimePoint(taxi_id=np.int64(161), name='start shift', time=0)
TimePoint(taxi_id=np.int64(831), name='start shift', time=0)
TimePoint(taxi_id=np.int64(993), name='start shift', time=0)
TimePoint(taxi_id=np.int64(965), name='start shift', time=0)
TimePoint(taxi_id=np.int64(433), name='start shift', time=0)
TimePoint(taxi_id=np.int64(553), name='start shift', time=0)
TimePoint(taxi_id=np.int64(763), name='start shift', time=0)
TimePoint(taxi_id=np.int64(926), name='start shift', time=0)
TimePoint(taxi_id=np.int64(421), name='start shift', time=0)
TimePoint(taxi_id=np.int64(550), name='start shift', time=0)
TimePoint(taxi_id=np.int64(591), name='start shift', time=0)
TimePoint(taxi_id=np.int64(760), name='start shift', time=0)
TimePoint(taxi_id=np.int64(827), name='start shift', time=0)
TimePoint(taxi_id=np.int64(166), name='start shift', time=0)
TimePoint(taxi_id=np.int